In [ ]:
!pip install pip install anvil-uplink

In [ ]:
# Importing Libraries
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop

import matplotlib.pyplot as plt
import anvil.server

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Init
text_df = pd.read_csv("/content/gdrive/My Drive/Data_berita.csv")

text = list(text_df.content.values)
joined_text = ' '.join(text)

partial_text = joined_text[:250407]

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(partial_text.lower())

unique_tokens = np.unique(tokens)
unique_tokens_index = {token: idx for idx, token in enumerate(unique_tokens)}

n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words) :
  input_words.append(tokens[i:i + n_words])
  next_words.append(tokens[i + n_words])

x = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

for i, words in enumerate(input_words) :
  for j, word in enumerate(words) :
    x[i, j, unique_tokens_index[word]] = 1
  y[i, unique_tokens_index[next_words[i]]] = 1

In [ ]:
# Train the model
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
layers = 3
rep = layers - 2
if rep > 2 :
    for l in range(rep) :
        model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])
history = model.fit(x,y,batch_size = 128, epochs = 200, shuffle = True)
model.save('mymodel.h5')

In [ ]:
# Re-train the model
# del model
# model = load_model('mymodel.h5')
# history = model.fit(x,y,batch_size = 128, epochs = 50, shuffle = True)
# model.save('mymodel.h5')

In [ ]:
# Draw the Graph

accuracy_list = history.history['accuracy']
loss_list = history.history['loss']

# Create x-axis values (epochs)
epochs = range(1, len(accuracy_list) + 1)

# Plot epoch vs accuracy
plt.plot(epochs, accuracy_list, 'b', label='Accuracy')
plt.title('Epoch vs Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Create x-axis values (epochs)
epochs = range(1, len(loss_list) + 1)

# Plot epoch vs accuracy
plt.plot(epochs, loss_list, 'b', label='Loss')
plt.title('Epoch vs Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Predict Next word

def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    x = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        if word in unique_tokens_index:
            x[0, i, unique_tokens_index[word]] = 1

    predictions = model.predict(x)[0]
    top_n_indices = np.argpartition(predictions, -n_best)[-n_best:]
    top_n_words = [unique_tokens[i] for i in top_n_indices]
    top_n_weights = [predictions[i] for i in top_n_indices]

    # Sort the predicted words based on weights in descending order
    sorted_words_weights = sorted(zip(top_n_words, top_n_weights), key=lambda x: x[1], reverse=True)
    sorted_words, sorted_weights = zip(*sorted_words_weights)

    res_arr = []
    # Print the predicted words and their weights
    for word, weight in zip(sorted_words, sorted_weights):
        res_arr.append(f"{word} - Weight: {weight * 100} %")
        # print(f"{word} - Weight: {weight * 100} %")

    return res_arr

In [ ]:
# Loading the model

model = load_model("/content/gdrive/My Drive/mymodel.h5")

In [ ]:
anvil.server.connect('server_LVEBX45C7W3XPYEU7HNV5YPO-EMOCVIU5JIAIFHFJ')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
@anvil.server.callable
def predict_word(input_text) :
  return '\n'.join(predict_next_word(input_text,5))

In [ ]:
anvil.server.wait_forever()

1/1 [==============================] - 0s 41ms/step


KeyboardInterrupt: ignored